In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
1.将survey_time数据提取出月份、日期、小时三个特征
2.调参：
    1.n_estimators=100, 0.5222291797119599
    2.reg_lambda=10， 0.5203506574827802
    3.learning_rate=0.1， 0.5172197871008141

"""

import xgboost as xgb
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data_dir = '/Users/changqingli/pycharm_projects/algorithms/weekend_content/'
train_data_file = 'data/happiness_train_abbr.csv'
test_data_dile = 'data/happiness_test_abbr.csv'

In [3]:
df_train = pd.read_csv(data_dir+train_data_file)
df_test = pd.read_csv(data_dir+test_data_dile)
df_train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,...,family_income,family_m,family_status,house,car,marital,status_peer,status_3_before,view,inc_ability
0,1,4,1,12,32,59,2015/8/4 14:18,1,1959,1,...,60000.0,2,2,1,2,3,3,2,4,3
1,2,4,2,18,52,85,2015/7/21 15:04,1,1992,1,...,40000.0,3,4,1,2,1,1,1,4,2
2,3,4,2,29,83,126,2015/7/21 13:24,2,1967,1,...,8000.0,3,3,1,2,3,2,1,4,2
3,4,5,2,10,28,51,2015/7/25 17:33,2,1943,1,...,12000.0,3,3,1,1,7,2,1,3,2
4,5,4,1,7,18,36,2015/8/10 9:50,2,1994,1,...,-2.0,4,3,1,1,1,3,2,3,-8


In [4]:
df_train = df_train.drop('id', 1)
df_train = df_train[df_train['happiness'] != -8]
df_train = df_train[pd.to_datetime(df_train.survey_time).dt.month != 2]


In [5]:
df_train['month'] = pd.to_datetime(df_train.survey_time).dt.month
df_train.head()

,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,religion,...,family_m,family_status,house,car,marital,status_peer,status_3_before,view,inc_ability,month
0,4,1,12,32,59,2015/8/4 14:18,1,1959,1,1,...,2,2,1,2,3,3,2,4,3,8
1,4,2,18,52,85,2015/7/21 15:04,1,1992,1,1,...,3,4,1,2,1,1,1,4,2,7
2,4,2,29,83,126,2015/7/21 13:24,2,1967,1,0,...,3,3,1,2,3,2,1,4,2,7
3,5,2,10,28,51,2015/7/25 17:33,2,1943,1,1,...,3,3,1,1,7,2,1,3,2,7
4,4,1,7,18,36,2015/8/10 9:50,2,1994,1,1,...,4,3,1,1,1,3,2,3,-8,8


In [6]:
df_train['day'] = pd.to_datetime(df_train.survey_time).dt.day


In [6]:
df_train['hour'] = pd.to_datetime(df_train.survey_time).dt.hour



In [7]:
df_train.head()

,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,religion,...,family_status,house,car,marital,status_peer,status_3_before,view,inc_ability,month,hour
0,4,1,12,32,59,2015/8/4 14:18,1,1959,1,1,...,2,1,2,3,3,2,4,3,8,14
1,4,2,18,52,85,2015/7/21 15:04,1,1992,1,1,...,4,1,2,1,1,1,4,2,7,15
2,4,2,29,83,126,2015/7/21 13:24,2,1967,1,0,...,3,1,2,3,2,1,4,2,7,13
3,5,2,10,28,51,2015/7/25 17:33,2,1943,1,1,...,3,1,1,7,2,1,3,2,7,17
4,4,1,7,18,36,2015/8/10 9:50,2,1994,1,1,...,3,1,1,1,3,2,3,-8,8,9


In [8]:
df_test['month'] = pd.to_datetime(df_test.survey_time).dt.month
df_test['day'] = pd.to_datetime(df_test.survey_time).dt.day
df_test['hour'] = pd.to_datetime(df_test.survey_time).dt.hour
df_train = df_train.drop('survey_time', 1)
df_test = df_test.drop('survey_time', 1)




In [9]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_train_train = df_train.sample(frac=0.8, random_state=0, axis=0)
df_train_validation = df_train[~df_train.index.isin(df_train_train.index)]


In [10]:
len(df_train_validation)

1597

In [11]:
tr_data = df_train_train.as_matrix()
va_data = df_train_validation.as_matrix()
te_data = df_test.as_matrix()
tr_data

/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


array([[ 3.,  1.,  1., ..., -8.,  9., 10.],
       [ 4.,  1., 12., ...,  2.,  8., 15.],
       [ 5.,  1., 21., ...,  2.,  7., 16.],
       ...,
       [ 3.,  2., 29., ...,  3.,  7., 19.],
       [ 5.,  1.,  1., ...,  2.,  9.,  9.],
       [ 4.,  2., 18., ..., -8.,  7., 12.]])

In [12]:
tr_x_data = tr_data[:, 1:]
tr_y_data = tr_data[:, 0]

va_x_data = va_data[:, 1:]
va_y_data = va_data[:, 0]

In [13]:
len(tr_x_data[0])

41

In [15]:
mod = xgb.XGBRegressor(max_depth=6,
                       learning_rate=0.1,
                       n_estimators=100,
                       objective='reg:squarederror',
                       booster='gbtree',
                       gamma=5,
                       reg_lambda=10)

In [16]:
mod.fit(tr_x_data, tr_y_data)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=5,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=10, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [17]:
va_pred_data = np.round(mod.predict(va_x_data))
np.mean((va_pred_data - va_y_data)*(va_pred_data - va_y_data))

0.5309956167814652

In [18]:
pred_data = te_data[:, 1:]
pred_res = np.round(mod.predict(pred_data))

import csv
result = 'submission_908_xgb_tiaocan.csv'
with open(data_dir+result, 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['id', 'happiness'])
    for i in range(len(te_data)):
        writer.writerow([int(te_data[i][0]), int(pred_res[i])])

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41']
training data did not have the following fields: f41

In [ ]:
tr_x_data
tr_y_data